## 异常检测
高斯模型来检测数据集中未标记的示例是否应被视为异常

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat
# 获取数据
data = loadmat('/home/kesci/input/andrew_ml_ex85033/ex8data1.mat')
X = data['X']
X.shape